#Sticker Creation

In [ ]:
!pip install --upgrade google-genai pillow rembg
!pip install --upgrade onnxruntime
!pip install python-dotenv

### `KEYS`

In [1]:
# Multiple OpenAI API Keys for parallel processing
API_KEYS = [ # 3 API keys
            "API KEY 1",
             "API KEY 2",
             "API KEY 3"
]

# Stickerverse


In [ ]:
import os
import random
import base64
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from openai import OpenAI
from PIL import Image
from io import BytesIO
from rembg import remove
from google.colab import files
from IPython.display import display, Javascript
from google.colab.output import eval_js
import time

In [ ]:
# Create OpenAI clients for each API key
clients = [OpenAI(api_key=key) for key in API_KEYS]

# Camera capture via JS
def capture_photo(filename='photo.jpg', quality=0.9):
    js_code = """
    async function takePhoto(quality) {
        const div = document.createElement('div');
        const video = document.createElement('video');
        const btn = document.createElement('button');
        btn.textContent = '📸 Capture';
        div.appendChild(video);
        div.appendChild(btn);
        document.body.appendChild(div);
        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        video.srcObject = stream;
        await video.play();
        await new Promise(resolve => btn.onclick = resolve);
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getTracks().forEach(track => track.stop());
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
    }
    """
    display(Javascript(js_code))
    data = eval_js("takePhoto(%f)" % quality)
    binary = base64.b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    print(f"Saved: {filename}")
    return filename

# Spelling validation function using OpenAI
def validate_and_correct_spelling(text):
    try:
        spelling_prompt = f"""
        Please check the spelling and grammar of the following text and return ONLY the corrected version.
        Do not add explanations, comments, or change the meaning. Just fix spelling and basic grammar errors.
        Keep the same tone, style, and language (including Hindi/English mix if present).

        Text to check: "{text}"

        Return only the corrected text:
        """

        response = clients[0].chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": spelling_prompt}],
            max_tokens=100,
            temperature=0.1  # Low temperature for consistent corrections
        )

        corrected_text = response.choices[0].message.content.strip()

        # Remove quotes if the LLM added them
        if corrected_text.startswith('"') and corrected_text.endswith('"'):
            corrected_text = corrected_text[1:-1]

        # Log if corrections were made
        if corrected_text != text:
            print(f"Spelling corrected: '{text}' → '{corrected_text}'")

        return corrected_text

    except Exception as e:
        print(f"Spelling validation failed: {str(e)}. Using original text.")
        return text

# Prompt builder for sticker with spelling validation
def build_prompt(text, style_variant):
    # Validate and correct spelling first
    corrected_text = validate_and_correct_spelling(text)

    base_prompt = (
        f"Create an ULTRA-HIGH QUALITY WhatsApp sticker with the style: {style_variant}. "
        "OUTPUT REQUIREMENTS: "
        "- Resolution: 1024x1024px (1:1 aspect ratio), transparent PNG, clean 8px die-cut white border "
        "- Consistent framing: subject centered, 24px padding from edges, balanced composition "
        "- CRISP, sharp details with professional sticker finish and high-quality rendering "
        "- Preserve the subject's ORIGINAL facial identity and proportions (do NOT morph or alter the face) "
        "- Maintain natural facial features, expressions, and skin tone accurately "
        "- For caricature: exaggerate features only slightly while keeping full recognition intact "
        "- Consistent color grading: vibrant but realistic (avoid oversaturation) "
        "- Perfect square format with subject filling most of the frame without cropping important parts "
        "TEXT REQUIREMENTS: "
        f"- Add EXACT text: '{corrected_text}' (NO spelling changes, NO extra words, NO emojis, NO decorative glyphs) "
        "- Text must be in a large, bold, comic-style font with 4-5px solid black outline for maximum readability "
        "- Use high-contrast colors (neon yellow, hot pink, electric blue, lime green) for perfect text visibility "
        "- Place text CLEARLY in empty space (top or bottom zone preferred), never overlapping the face/body "
        "- Ensure perfect spelling, letter clarity, and NO stylistic distortions of characters "
        "- The text has been pre-validated for spelling accuracy - use it EXACTLY as provided "
        "STYLE INSTRUCTIONS: "
        f"- Apply '{style_variant}' effect with consistent high quality, sharpness, and rich texture "
        "- Maintain professional sticker quality with smooth edges and clean finish "
        "STRICT RULES: "
        "- Do NOT hallucinate extra elements or props unless they directly support the humor or theme of the provided text "
        "- Do NOT crop parts of the head/face or cut text "
        "- Do NOT stylize or replace the provided text "
        "- Stick to consistent framing and proportions across all stickers "
        "- Ensure facial features are sharp and proportions natural "
        "- Do not hallucinate extra elements, do not change words, do not generate decorative text "
        "- Do not add unnecessary objects or backgrounds that distract from the subject "
        "- CRITICAL: Use the exact spelling provided in the text - it has been pre-validated for accuracy "
        "- Ensure 1024x1024 pixel output with perfect square dimensions "
        f"""### MOST IMPORTANT:
           - Match the subject's facial expression, body language, and overall mood to perfectly reflect the emotion, tone, and sentiment implied by the provided text: '{corrected_text}'
           - The {style_variant} should enhance and amplify the emotional expression that fits the text
           - If the text is funny/humorous, show a happy, laughing, or smirking expression
           - If the text is serious/dramatic, show a serious, intense, or focused expression
           - If the text is sad/emotional, show a sad, concerned, or empathetic expression
           - If the text is confident/boastful, show a confident, proud, or cocky expression
           - If the text is tired/exhausted, show a tired, weary, or frustrated expression
           - The facial expression should be the PRIMARY way the sticker communicates the text's meaning
           - Make sure to use the {corrected_text} as it is without making even a slightest change in spelling!!"""
    )
    return base_prompt

# Updated Style variants for sticker look (2 detailed caricature + 1 detailed Pixar)
STYLE_VARIANTS = [
    "Transform into detailed caricature with slightly exaggerated facial features (bigger eyes, expressive eyebrows, enhanced smile/frown) while maintaining full recognizability. Use bold cartoon-style outlines, vibrant colors, and comic book shading. Focus on amplifying natural expressions and personality traits through subtle feature enhancement. Keep skin tones natural but with cartoon-like smoothness and clarity.",
    "Transform into expressive caricature with enhanced personality features (accentuated expressions, lively eyes, dynamic hair, emphasized facial characteristics) while preserving the subject's identity. Apply clean cartoon aesthetics with rich colors, smooth gradients, and professional illustration quality. Emphasize emotional expressiveness through slightly exaggerated but tasteful feature enhancement.",
    "Transform into high-quality Pixar-style 3D animated character with smooth, polished surfaces, soft realistic lighting, and Disney-Pixar aesthetic. Maintain photorealistic facial structure while applying the signature Pixar character treatment: clean smooth skin, expressive large eyes, perfect hair rendering, and warm cinematic lighting. Use Pixar's signature color palette with rich, saturated but natural tones. Apply subtle stylization that keeps the subject completely recognizable while giving them the premium Pixar animated movie character appearance with professional 3D rendering quality."
]

# 🎭 PHRASE CATEGORIES (with spell-checked phrases)
PHRASE_CATEGORIES = {
    "corporate": [
        "Another meeting? May the force be with you!",
        "Monday blues activated!",
        "This could have been an email, boss!",
        "Client bola - picture abhi baaki hai!",
        "Boss aaya, tension leke aaya!",
        "Work from home, par kaam double hai!",
        "Promotion letter? Beta tumse na ho payega!",
        "Appraisal ke naam pe dhokha hai yeh!",
        "Deadline sunte hi jaan nikal jaati hai!",
        "Team bonding = free pizza!",
        "Boss entry: Mogambo khush hua!",
        "Status update? Bhai, zinda hoon bas!",
        "Coffee nahi toh coding nahi!",
        "Jab tak hai jaan, tab tak hai kaam!",
        "Client feedback: Dil garden garden ho gaya!",
        "Overtime ka asli naam slavery hai!",
        "HR email aaya? Bas samajh ja, kuch gadbad hai!",
        "Meeting mein hawa bharna mera talent hai!",
        "Kabhi kabhi kuch jeetne ke liye kuch weekend khona padta hai!",
        "Quarter end: Koi to mujhe bachao!",
        "Late night deploy? Bhai, mujhe ghar jaana hai!",
        "Annual audit: Sab mile hue hain!",
        "Manager bole: ROI looks promising!",
        "Bandwidth issues = kaam mat do bhai!",
        "Boss ki smile = sabse bada danger signal!",
        "Client call ka intzaar to Don bhi krta hai!",
        "HR ke samne pyaar se bolo varna salary kaat lega!",
        "Beta tumse presentation na ho payega!",
        "Slack pe green light fake hai!",
        "Zoom call ka dress code: upar formal, neeche pajama!",
        "PowerPoint mein power kam, point zyada hai!",
        "Spreadsheet mein emotions dikh rahe hain!",
        "Monday motivation? Sirf chhutti ka calendar!",
        "Team meeting ka main agenda: free snacks!",
        "Boss ka motivational speech sunke toh neend bhi aa jaati hai!",
        "Lunch break ka asli matlab - fridge ke samne khade rehna!",
        "Wifi slow hai, tension double hai!",
        "Office ka AC matlab - ek office, do mausam!",
        "Meeting start hone se pehle coffee ka number badhao!",
        "Boss ke jokes par hasi mandatory, chahe kuch samajh na aaye!",
        "Office politics mein survivor bano, game of thrones style!",
        "Printer kabhi kaam karega? Yeh toh sirf humari ummeed jagata hai!",
        "Lunch toh mila, par plate mein sirf hawa thi!",
        "Abhi toh weekend door hai, par boss ki aankhen Monday ki taraf hain!",
        "Open the kimono: Share your coffee stash!",
        "Trim the fat: Cut the chit-chat in meetings!",
        "Drink the Kool-Aid: Believe in unlimited vacation (terms apply)!",
        "Herding cats: Managing a team on a Friday afternoon!",
        "Boil the ocean: Trying to reply all emails in one day!",
        "Machine learning ka matlab hai, data se dosti karna!",
        "AI bot: Main aapki har problem ka solution, ya pizza order ho sakta hoon!",
        "Data scientist ki life: Data clean kro, coffee pilao!",
        "Model train karna hai ya train miss karna hai?",
        "Overfitting? Matlab model ko zyada pyaar kar diya!",
        "Prediction mein toh har koi expert hai, jab result aata hai tab pata chalta hai!",
        "Data scientist ka favourite game: Tidy up the dataset!",
        "AI ka asli magic? Data ko samajhna, aur phir confuse karna!",
        "Big data matlab bada headache!",
        "Neural network: Dimag se zyada complex, par human se kam samajhdar!",
        "Train-test split karte jao, results confuse karte jao!",
        "Null values se dosti karni padti hai!",
        "Algorithm samajhna mushkil hai, par coffee samajhna asaan!",
        "Kaggle competition: Jitna data, utni neend kam!",
        "Model ko downgrade nahi, bas patience badhao!",
        "Data leaks se bachaao, warna boss pata laga lega!",
        "AI experts ki language: Hum bas machine se baat karte hain!",
        "Debugging matlab: Kahaan galti hui? Bas ussi ko khojte raho!",
        "Deep learning ka funda: Zyada layers, zyada drama!",
        "Python ki duniya mein sab kuch possible hai, except weekend off!",
        "Clustering matlab kaam ko groups mein baantna, par ghar ke kaam nahi!",
        "Data scientist ka secret weapon: Coffee, aur thodi si magic!",
        "Jab tak data sahi nahi, model bhi nahi chalega!",
        "AI le lo, par boss ke mood ka model kaun train kare?",
        "AI le aao, par promotion toh boss ki marzi se hi milegi!",
        "Feature selection: Bas wahi feature lo jo boss ko pasand aaye!"
    ],
    "bollywood": [
        "Mogambo khush hua!",
        "Kitne aadmi the?",
        "Picture abhi baaki hai mere dost!",
        "Pushpa, I hate tears!",
        "Beta tumse na ho payega!",
        "Rishte mein toh hum tumhare baap lagte hain!",
        "Don ka intezaar toh baarish bhi karti hai!",
        "Kabhi kabhi kuch jeetne ke liye kuch haarna padta hai!",
        "Mere paas maa hai!",
        "Kuch kuch hota hai!",
        "Bade bade deshon mein choti choti baatein hoti rehti hain!",
        "Tension lene ka nahi, dene ka!",
        "Zindagi na milegi dobara!",
        "Hum saath saath hain!",
        "Taareek pe taareek milti rahi hai!",
        "Rahul, naam toh suna hoga!",
        "Tum mujhe tang karne lage ho!",
        "Haan main pagal hoon!",
        "Babumoshai, zindagi badi honi chahiye, lambi nahi!",
        "Thappad se darr nahi lagta sahab, pyaar se lagta hai!",
        "Dost fail ho jaye toh dukh hota hai, par jab dosti hi fail ho jaaye tab?",
        "Mujhse shaadi karogi?",
        "Khamoshi se shor machta hai!",
        "Jab tak hai jaan, tab tak hai kaam!",
        "Dil to pagal hai!",
        "Kisi ka bhai kisi ki jaan!",
        "Main apni favourite hoon!",
        "Basanti, in kutton ke samne mat naachna!",
        "Aaj mere paas gaadi hai, bangla hai, paisa hai… tumhare paas kya hai?",
        "Babuji ne kaha gaon chhod do, sab ne kaha… chhod diya!",
        "Mere Karan Arjun aayenge!",
        "Yeh Dhakkan mere se tez dimaag kaun hai?",
        "Mujhko toh bas murgi ki tarah yaad rehna hai!",
        "Pyaar dosti hai, aur dosti mein bill baantne ki baat nahi hoti!",
        "Movie dekh ke bola, popcorn lekar aao, par popcorn khatam hai!",
        "Cinema hall mein lip balm dhundhne aa gaya hoon main!",
        "Yeh Baburao ka style hai!",
        "Utha le re baba, utha le!",
        "Tumhara naam kya hai Basanti?",
        "Iska toh plan hi ulta hai!",
        "Baap hamesha baap hota hai, beta hamesha beta hota hai!"
    ],
    "hollywood": [
        "I'll be back - when WiFi works!",
        "May the force be with you - on Monday!",
        "Houston, we have a problem!",
        "Show me the money!",
        "You can't handle the truth!",
        "I see dead people - in this meeting!",
        "Frankly my dear, I don't give a damn!",
        "We're gonna need a bigger boat!",
        "Say hello to my little friend - coffee!",
        "You talking to me?",
        "I feel the need... the need for speed!",
        "Nobody puts Baby in a corner!",
        "Roads? Where we're going, we don't need roads!",
        "Keep your friends close, but your charger closer!",
        "I'm gonna make him an offer he can't refuse!",
        "You complete me!",
        "I'm the king of the world!",
        "Life is like a box of chocolates!",
        "After all, tomorrow is another day!",
        "Here's Johnny!",
        "E.T. phone home!",
        "Elementary, my dear Watson!",
        "Hasta la vista, baby!",
        "You can't sit with us!",
        "With great power comes great responsibility!",
        "I am your father!",
        "Yippee-ki-yay - it's finally Friday!",
        "Why so serious? Because it's Monday again!",
        "To infinity… and the weekend nap!",
        "Winter is coming… so is my coffee order!",
        "I volunteer as tribute… for the office snacks!",
        "Just keep swimming… through the Zoom calls!",
        "I'm not a superhero, but I pretend in meetings!",
        "This is my happy face… just kidding, it's Monday!",
        "May the coffee be strong and emails be short!",
        "I've got 99 problems but a coffee ain't one!",
        "If you want to survive Monday, master the art of the nap!",
        "You sit on a throne of lies!",
        "It's just a flesh wound!",
        "Hello, my name is Inigo Montoya. You killed my father. Prepare to die!",
        "Looks like I picked the wrong week to quit sniffing glue!",
        "You're killing me, Smalls!"
    ],
    "tollywood": [
        "Pushpa flower nahi, fire hai!",
        "Baahubali ki tarah sab jeet lenge!",
        "Arjun Reddy mode on!",
        "Magadheera vibes la rahe hain!",
        "Geetha Govindam style awkward silences!",
        "Ala Vaikunthapurramuloo feels only!",
        "Rangasthalam ka swag alag hi level ka hai!",
        "Temper control nahi ho raha bhai!",
        "Pokiri dialogue maar ke impress karenge!",
        "Mirchi jaisa spice hona chahiye life mein!",
        "Chatrapathi energy needed in office!",
        "Simhadri roar jab kaam koi chura le!",
        "Yamadonga comedy = life story!",
        "Jersey spirit = kabhi haar nahi maanenge!",
        "Businessman level plans chahiye career mein!",
        "Sye ki team spirit laani padegi!",
        "Okkadu solo fight hi karni padti hai!",
        "Dookudu punch dialogues are life!",
        "Athadu jaise precision chahiye meetings mein!",
        "Jalsa ho jaaye boss, project ke baad!",
        "Khaidi No 150 trials in office!",
        "Seethamma Vakitlo Sirimalle Chettu - longest title ever!",
        "Srimanthudu vibes = generosity in office treats!",
        "Attarintiki Daredi - ghar wapas jaane ka plan!",
        "Annavaram dedication = weekend goals!",
        "Pushpa style: main jhukega nahi!",
        "Evaru mystery solving mode on!",
        "Bimbisara time travel ideas in meetings!",
        "DJ Tillu swag = Friday vibes!",
        "Mass Maharaja energy, boss!",
        "Kabaddi match mein bhi strategy zaroori hai!",
        "Office mein tension? Simham ki tarah roar karo!",
        "Break room mein snacks ka raaj hai!",
        "Deadline se pehle coffee, baad mein baraat!",
        "Kaam adhura toh drama full!"
    ],
    "sports": [
        "Ball de na, World Cup nahi jeetna mujhe!",
        "Bhai, pitch pe baatein nahi, shots khel!",
        "Chal bhaag, stadium tera baap ka nahi hai!",
        "Garden mein mat ghumo, yeh ground hai!",
        "Yaha kya dekh raha hai, teri GF fielding kar rahi hai kya?",
        "Umpire se panga nahi, varna ghar se bula lenge mummy ko!",
        "Bhai, six maarne gaya tha ya shopping krne?",
        "Goal maar bhai, shaadi nahi karni usse!",
        "Dekh bhai, ball miss mat karna, yaha crowd judgemental hai!",
        "Trophy lene aaye ho ya selfie?",
        "Shot maar, coaching centre nahi chal raha yaha!",
        "Ye cricket hai, daily soap nahi!",
        "Captain tu hai ya DJ bajane aaya hai?",
        "Bhai, net practice mein mat hero ban, match mein dikha!",
        "Football nahi hai yeh, kabaddi nahi khelni!",
        "Bouncer se darr gaya kya? Yeh toh trial run tha!",
        "Bhai, full toss ko respect mat de, maar de seedha boundary pe!",
        "Bowling kar raha hai ya yoga practice?",
        "Chal bhai, over ho gaya, ja thoda paani pi!",
        "Bhai, body language se batting nahi hoti!",
        "Aise mat khel, log sochenge family picnic pe aaye ho!",
        "Dekh bhai, fitness ground pe hoti hai, DP pe nahi!",
        "Match khel raha hai ya poetry sunane aaya hai?",
        "Bhai, match jeet le pehle, fir attitude dikhana!",
        "Scorecard dekh ke rona mat, kal ka match jeet le!",
        "Cricket ka love story: pitch pe hi hota hai!",
        "Penalty miss kiya? Public yaad rakhegi!",
        "Warm-up zyada, performance kam!",
        "Referee se panga = red card express!",
        "Half-time snacks > full-time score!",
        "Fan selfie mein bhi focus important hai!",
        "Coach ke gusse se zyada dar TV umpire ka hota hai!",
        "Captain ki strategy: poori team confuse!",
        "Innings ke baad chai time mandatory hai!",
        "Match mein cheer karne wali team zyada loud hoti hai!",
        "When the fan catches the ball but drops his food twice!",
        "The escalator slide gone wrong after darts!",
        "JR Smith thinking they were ahead in the finals!",
        "The casual shooter with hand in pocket winning silver!",
        "Double groin hit in cricket with mic on!"
    ],
    "memes": [
        "Bhagwan bharose!",
        "Main thak gaya hoon!",
        "Beta tumse na ho payega!",
        "Mujhe ghar jaana hai!",
        "Bhai calm reh!",
        "Aukaat ke bahar mat ja!",
        "Ye sab doglapan hai!",
        "Aree bhai, maaro mujhe maaro!",
        "Kuch bhi!",
        "Mujhe farak nahi padta!",
        "Scene serious hai bhai!",
        "Tumse na ho payega!",
        "Paisa hi paisa hoga!",
        "Kyu puch raha hai bhai?",
        "Ye bhi theek hai!",
        "Aaj kuch toofani karte hain!",
        "Chal nikal!",
        "Mood off hai!",
        "Pawri ho rahi hai!",
        "Feeling cute, might delete later!",
        "Niyat kharab hai!",
        "Kitna dramatize karega bhai?",
        "Matlab kuch bhi!",
        "Samajh raha hai na bhai?",
        "Sab moh maya hai!",
        "Lag raha hai April Fool ban gaya!",
        "Mereko kya!",
        "Bhai, risk hai!",
        "Choti bacchi ho kya?",
        "Internet kabhi mazak nahi karta!",
        "Tu jaanta nahi mera baap kaun hai!",
        "Mujhe gussa kyu aata hai?",
        "Bohot hard!",
        "Jhakaas!",
        "Dil garden garden ho gaya!",
        "Sawaal hi nahi uthta!",
        "Bhai ki tareef kam hai!",
        "Kya hi bolun!",
        "Ye sab setting hai!",
        "Bhai kya kar raha hai bhai!",
        "Jaldi bol, kal subah Panvel nikalna hai!",
        "Apun ko logic nahi emotion samajhta hai!",
        "Bas kar pagle, rulaayega kya!",
        "Bilkul risks nahi lene ka!",
        "Yeh bik gayi hai gormint!",
        "Mujhse na ho payega!",
        "Shakal dekh ke lagta hai Monday hai!",
        "Phone ka charger meri jaan se bhi zyada precious hai!",
        "Meme banao, life chhodo!",
        "Mood kharab = maggi therapy!",
        "Selfie le le re!",
        "Zoom call pe 'You're on mute' ka syndrome!",
        "Internet slow hai toh blame kar do router ko!",
        "Aaj toh ghar se office ka combo lag raha hai!",
        "Mujhe toh bas memes hi samajhte hain!",
        "Kitna bhi socho, kuch bhi theek nahi hota!",
        "WiFi chala gaya toh life ruk gayi jaise!",
        "Phone ne aaj tod diya mera patience ka record!",
        "Planned to do work, ended up planning to nap!",
        "Kabhi-kabhi laga jaise doston ke groups hi sab tension hain!",
        "Aaj kal ke toh memes hi asli psychologist hain!",
        "Mike Pence fly during debate!",
        "Tiger King anything!",
        "Nevada slow election count!",
        "Titanic sinking for 2020!",
        "Hope this email finds you well in 2020!",
        "Iska to game bhajana padega!",
        "Uthale re deva, uthale mereko nahi isko!",
        "25 din me paisa double!",
        "Ruk tera to game bajana padega!"
    ]
}

# Generate single sticker using OpenAI GPT-image-1 with specific client (WITH TIMING)
def generate_single_sticker(input_path, output_path, text, style_variant, client_idx):
    try:
        start_time = time.time()
        thread_id = threading.current_thread().name
        print(f"[START] Thread-{thread_id}: API-{client_idx+1} generating {style_variant[:30]}... at {time.strftime('%H:%M:%S', time.localtime(start_time))}")

        prompt = build_prompt(text, style_variant)

        result = clients[client_idx].images.edit(
            model="gpt-image-1",
            image=[open(input_path, "rb")],
            prompt=prompt,
            # input_fidelity="high"
            quality = 'medium'
        )

        image_base64 = result.data[0].b64_json
        image_bytes = base64.b64decode(image_base64)

        with open(output_path, "wb") as f:
            f.write(image_bytes)

        end_time = time.time()
        duration = end_time - start_time
        style_type = "Caricature" if "caricature" in style_variant.lower() else "Pixar"
        print(f"[DONE] Thread-{thread_id}: {style_type} saved as {output_path} | Duration: {duration:.2f}s | Text: '{text[:30]}...'")
        return True

    except Exception as e:
        print(f"[ERROR] API-{client_idx+1} failed: {str(e)}")
        return False

# NEW: Function to select between category and custom prompt
def select_category_or_custom():
    print("\nChoose your sticker text option:")
    print("1. Pick from phrase category (random selection)")
    print("2. Enter my own custom phrase")

    while True:
        try:
            choice = input("Choose option (1 or 2): ").strip()

            if choice == "1":
                return "category"
            elif choice == "2":
                return "custom"
            else:
                print("Invalid choice! Please enter 1 or 2.")

        except KeyboardInterrupt:
            print("\nGoodbye!")
            return None

# NEW: Function to get custom phrase from user
def get_custom_phrase():
    while True:
        phrase = input("\nEnter your custom sticker text (2-50 characters): ").strip()

        if len(phrase) < 2:
            print("Too short! Please enter at least 2 characters.")
            continue
        elif len(phrase) > 50:
            print("Too long! Please keep it under 50 characters.")
            continue
        else:
            print(f"Custom phrase accepted: '{phrase}'")
            return phrase

# NEW: Create stickers with custom phrase (all 3 styles use the same custom text)
def create_custom_stickers_parallel(photo_file, custom_text):
    print(f"\nCreating 3 stickers with your custom phrase: '{custom_text}'")
    print("   • Style 1: Caricature #1")
    print("   • Style 2: Caricature #2")
    print("   • Style 3: Pixar Animation")

    # Map futures to their info
    tasks_info = {}

    with ThreadPoolExecutor(max_workers=3, thread_name_prefix="CustomSticker") as executor:
        start_time = time.time()
        print(f"\n[PARALLEL START] Submitting 3 API calls SIMULTANEOUSLY at {time.strftime('%H:%M:%S', time.localtime(start_time))}")

        # Submit ALL tasks at once (non-blocking) - all using the same custom text
        for idx, style_variant in enumerate(STYLE_VARIANTS):
            output_name = f"custom_sticker_{idx+1}.png"
            future = executor.submit(generate_single_sticker, photo_file, output_name, custom_text, style_variant, idx)
            tasks_info[future] = {
                'output_name': output_name,
                'text': custom_text,
                'style_variant': style_variant,
                'client_idx': idx,
                'submit_time': time.time()
            }

        print("All 3 API requests submitted! Processing as they complete...")

        completed = 0
        completion_times = []

        # Process results as they complete
        for future in as_completed(tasks_info.keys(), timeout=180):
            try:
                success = future.result()
                task_info = tasks_info[future]

                if success:
                    completed += 1
                    completion_time = time.time()
                    completion_times.append(completion_time)
                    duration = completion_time - task_info['submit_time']
                    style_type = "Caricature" if "caricature" in task_info['style_variant'].lower() else "Pixar"

                    print(f"[{completed}/3] {style_type} completed: {task_info['output_name']} "
                          f"(API-{task_info['client_idx']+1}, {duration:.1f}s)")
                else:
                    print(f"Failed: {task_info['output_name']}")

            except Exception as e:
                task_info = tasks_info[future]
                print(f"Error with {task_info['output_name']} (API-{task_info['client_idx']+1}): {str(e)}")

        total_time = time.time() - start_time
        print(f"\n [FINAL RESULT] {completed}/3 custom stickers completed in {total_time:.1f} seconds!")

# UPDATED: Create 3 stickers in  PARALLEL (using as_completed)
def create_category_stickers_parallel(photo_file, category):
    if category not in PHRASE_CATEGORIES:
        print(f" Category '{category}' not found! Available: {list(PHRASE_CATEGORIES.keys())}")
        return

    # Choose 3 unique phrases for 3 stickers
    chosen_phrases = random.sample(PHRASE_CATEGORIES[category], 3)

    print(f" Selected phrases for {category.title()} category:")
    for i, phrase in enumerate(chosen_phrases, 1):
        style_type = "Caricature" if i <= 2 else "Pixar Animation"
        print(f"   {i}. [{style_type}] '{phrase}' → API Key {i}")

    # Map futures to their info
    tasks_info = {}

    with ThreadPoolExecutor(max_workers=3, thread_name_prefix="StickerGen") as executor:
        start_time = time.time()
        print(f"\n [PARALLEL START] Submitting 3 API calls SIMULTANEOUSLY at {time.strftime('%H:%M:%S', time.localtime(start_time))}")

        # Submit ALL tasks at once (non-blocking)
        for idx, (style_variant, text) in enumerate(zip(STYLE_VARIANTS, chosen_phrases)):
            output_name = f"{category}_sticker_{idx+1}.png"
            future = executor.submit(generate_single_sticker, photo_file, output_name, text, style_variant, idx)
            tasks_info[future] = {
                'output_name': output_name,
                'text': text,
                'style_variant': style_variant,
                'client_idx': idx,
                'submit_time': time.time()
            }

        print("All 3 API requests submitted! Processing as they complete...")
        print("   • API Key 1 → Caricature #1")
        print("   • API Key 2 → Caricature #2")
        print("   • API Key 3 → Pixar Animation")

        completed = 0
        completion_times = []

        # Process results as they complete (NOT in submission order)
        for future in as_completed(tasks_info.keys(), timeout=180):  # 3 minute total timeout
            try:
                success = future.result()  # This only waits until ANY future completes
                task_info = tasks_info[future]

                if success:
                    completed += 1
                    completion_time = time.time()
                    completion_times.append(completion_time)
                    duration = completion_time - task_info['submit_time']
                    style_type = "Caricature" if "caricature" in task_info['style_variant'].lower() else "Pixar"

                    print(f"[{completed}/3] {style_type} completed: {task_info['output_name']} "
                          f"(API-{task_info['client_idx']+1}, {duration:.1f}s) - '{task_info['text'][:30]}...'")
                else:
                    print(f"Failed: {task_info['output_name']}")

            except Exception as e:
                task_info = tasks_info[future]
                print(f"Error with {task_info['output_name']} (API-{task_info['client_idx']+1}): {str(e)}")

        total_time = time.time() - start_time
        print(f"\n[FINAL RESULT] {completed}/3 stickers completed in {total_time:.1f} seconds!")

        if len(completion_times) > 1:
            fastest_completion = min(completion_times) - start_time
            print(f"Parallel efficiency: Fastest completion in {fastest_completion:.1f}s")

# Image upload function
def upload_image():
    print("Please upload your image file...")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded!")
        return None

    filename = list(uploaded.keys())[0]
    print(f"Uploaded: {filename}")

    # Validate if it's an image
    try:
        img = Image.open(filename)
        img.verify()
        print(f"📸 Image verified: {img.format} {img.size}")
        return filename
    except Exception as e:
        print(f"Invalid image file: {str(e)}")
        return None

# Interactive image source selection
def select_image_source():
    print("Choose image source:")
    print("1. Capture from camera")
    print("2. Upload image file")

    while True:
        try:
            choice = input("Select option (1-2): ").strip()

            if choice == "1":
                return "camera"
            elif choice == "2":
                return "upload"
            else:
                print("Invalid choice! Please enter 1 or 2.")

        except KeyboardInterrupt:
            print("\nGoodbye!")
            return None

# Interactive category selection
def select_category():
    print("Available Categories:")
    categories = list(PHRASE_CATEGORIES.keys())
    for i, category in enumerate(categories, 1):
        print(f"{i}. {category.title()} ({len(PHRASE_CATEGORIES[category])} phrases)")

    while True:
        try:
            choice = input(f"\nSelect category (1-{len(categories)}) or enter category name: ").strip()

            # Check if it's a number
            if choice.isdigit():
                idx = int(choice) - 1
                if 0 <= idx < len(categories):
                    return categories[idx]
                else:
                    print("Invalid number! Please try again.")

            # Check if it's a category name
            elif choice.lower() in categories:
                return choice.lower()

            else:
                print("Invalid input! Please enter a number or category name.")

        except KeyboardInterrupt:
            print("\nGoodbye!")
            return None

# Function to validate all phrases in categories (run once to check)
def validate_all_phrases():
    """
    One-time function to validate and correct spelling in all phrase categories.
    Use this to clean up the phrase database.
    """
    print("Validating spelling in all categories...")

    for category, phrases in PHRASE_CATEGORIES.items():
        print(f"\nChecking {category} category...")
        for i, phrase in enumerate(phrases):
            corrected = validate_and_correct_spelling(phrase)
            if corrected != phrase:
                PHRASE_CATEGORIES[category][i] = corrected

    print("All phrases validated and corrected!")

# UPDATED: Main function with image source and category/custom selection
def sticker_from_camera():
    print("Multi-Category Sticker Generator with Custom Prompts (TRUE PARALLEL Processing)")
    print("Spelling guardrails activated!")
    print("Using 3 API keys for SIMULTANEOUS generation!")
    print("2 Caricature + 1 Pixar Animation per session")
    print("NEW: Custom prompt support added!")

    # Select image source
    image_source = select_image_source()
    if not image_source:
        return

    # Get image file
    if image_source == "camera":
        print("Preparing camera capture...")
        photo_file = capture_photo()
    else:  # upload
        photo_file = upload_image()

    if not photo_file:
        print("No image available. Exiting...")
        return

    # NEW: Select between category and custom prompt
    choice = select_category_or_custom()
    if not choice:
        return

    if choice == "category":
        # Original category flow
        category = select_category()
        if not category:
            return

        print(f"\nCreating {category.title()} stickers (2 caricature + 1 Pixar)...")
        create_category_stickers_parallel(photo_file, category)
        print(f"Completed your {category.title()} sticker set!")

    elif choice == "custom":
        # NEW: Custom prompt flow
        custom_phrase = get_custom_phrase()
        create_custom_stickers_parallel(photo_file, custom_phrase)
        print("Completed your custom sticker set!")

# Utility function to create stickers for all categories (with parallel processing)
def create_all_categories_stickers():
    print("Creating stickers for ALL categories...")

    # Select image source
    image_source = select_image_source()
    if not image_source:
        return

    # Get image file
    if image_source == "camera":
        photo_file = capture_photo()
    else:  # upload
        photo_file = upload_image()

    if not photo_file:
        print("No image available. Exiting...")
        return

    for category in PHRASE_CATEGORIES.keys():
        print(f"\nGenerating {category.title()} stickers...")
        create_category_stickers_parallel(photo_file, category)

    print(f"Completed sticker sets for all {len(PHRASE_CATEGORIES)} categories!")

# Quick functions for direct usage
def sticker_from_uploaded_image():
    """Generate stickers from uploaded image directly"""
    print("Sticker Generator - Upload Mode (TRUE PARALLEL)")
    print("Output: 1024x1024px Pixar + Caricature stickers")

    photo_file = upload_image()
    if not photo_file:
        return

    # NEW: Select between category and custom prompt
    choice = select_category_or_custom()
    if not choice:
        return

    if choice == "category":
        category = select_category()
        if not category:
            return
        print(f"Creating {category.title()} stickers...")
        create_category_stickers_parallel(photo_file, category)
        print(f"Completed your {category.title()} sticker set!")
    elif choice == "custom":
        custom_phrase = get_custom_phrase()
        create_custom_stickers_parallel(photo_file, custom_phrase)
        print("Completed your custom sticker set!")

def sticker_from_camera_direct():
    """Generate stickers from camera directly"""
    print("🎬 Sticker Generator - Camera Mode (TRUE PARALLEL)")
    print("📐 Output: 1024x1024px Pixar + Caricature stickers")

    photo_file = capture_photo()
    if not photo_file:
        return

    # NEW: Select between category and custom prompt
    choice = select_category_or_custom()
    if not choice:
        return

    if choice == "category":
        category = select_category()
        if not category:
            return
        print(f"Creating {category.title()} stickers...")
        create_category_stickers_parallel(photo_file, category)
        print(f"Completed your {category.title()} sticker set!")
    elif choice == "custom":
        custom_phrase = get_custom_phrase()
        create_custom_stickers_parallel(photo_file, custom_phrase)
        print("Completed your custom sticker set!")

# Function to add custom phrases to any category (with spelling validation)
def add_custom_phrases(category, phrases_list):
    if category in PHRASE_CATEGORIES:
        # Validate spelling of new phrases before adding
        validated_phrases = []
        for phrase in phrases_list:
            corrected_phrase = validate_and_correct_spelling(phrase)
            validated_phrases.append(corrected_phrase)

        PHRASE_CATEGORIES[category].extend(validated_phrases)
        print(f"Added {len(validated_phrases)} spell-checked phrases to {category.title()}")
    else:
        print(f"Category '{category}' not found!")

# NEW: Interactive menu for multiple functions
def interactive_menu():
    print("\nWhatsApp Sticker Generator - Interactive Menu")
    print("=" * 50)
    print("1. Create stickers (Camera/Upload + Category/Custom)")
    print("2. Camera mode only")
    print("3. Upload mode only")
    print("4. Generate ALL categories (bulk)")
    print("5. Validate all phrase spellings")
    print("6. Exit")

    while True:
        try:
            choice = input("\nSelect option (1-6): ").strip()

            if choice == "1":
                sticker_from_camera()
                break
            elif choice == "2":
                sticker_from_camera_direct()
                break
            elif choice == "3":
                sticker_from_uploaded_image()
                break
            elif choice == "4":
                create_all_categories_stickers()
                break
            elif choice == "5":
                validate_all_phrases()
                break
            elif choice == "6":
                print("Goodbye!")
                break
            else:
                print("Invalid choice! Please enter 1-6.")

        except KeyboardInterrupt:
            print("\nGoodbye!")
            break

# Main execution
if __name__ == "__main__":
    # Uncomment the line below to validate all existing phrases (run once)
    # validate_all_phrases()

    # Run the main workflow with custom prompt support
    sticker_from_camera()

    # Alternative: Run the interactive menu
    # interactive_menu()

    # Uncomment below to generate all categories at once
    # create_all_categories_stickers()


Multi-Category Sticker Generator with Custom Prompts (TRUE PARALLEL Processing)
Spelling guardrails activated!
Using 3 API keys for SIMULTANEOUS generation!
2 Caricature + 1 Pixar Animation per session
NEW: Custom prompt support added!
Choose image source:
1. Capture from camera
2. Upload image file
